In [1]:
import numpy as np
import random, math
import copy

Dimension = 4
qubits = 3
A = np.array([[-9, -2,-2,9], [-3, 10,-3,-3], [-8,6,-9,-8], [-8,8,10,9]]) # Row vectors
b = np.array([-29, 19, 30, -19]) # Column vectors
# x1 = q1 + 2q2 - 4q3

QM = np.zeros(((qubits+1)*Dimension, (qubits+1)*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        cef1 = pow(2,2*qubits)*pow(A[k][i],2)
        cef2 = pow(2,qubits+1)*A[k][i]*b[k]
        po2 = (qubits+1)*i + qubits 
        QM[po2][po2] = QM[po2][po2] + cef1 + cef2
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = (qubits+1)*i + l
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l in range (qubits):
            qcef = pow(2, l+qubits+1)*pow(A[k][i],2)
            po3 = (qubits+1)*i + l
            po4 = (qubits+1)*i + qubits 
            QM[po3][po4] = QM[po3][po4] - qcef
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = (qubits+1)*i + l1
                po2 = (qubits+1)*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                
### Second quadratic term ###                 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            qcef = pow(2, 2*qubits+1)*A[k][i]*A[k][j]
            po3 = (qubits+1)*i + qubits
            po4 = (qubits+1)*j + qubits
            QM[po3][po4] = QM[po3][po4] + qcef
            for l in range (qubits):
                qcef = pow(2, l+qubits+1)*A[k][i]*A[k][j]
                po5 = (qubits+1)*i + qubits 
                po6 = (qubits+1)*j + l
                QM[po5][po6] = QM[po5][po6] - qcef
                po7 = (qubits+1)*i + l
                po8 = (qubits+1)*j + qubits
                QM[po7][po8] = QM[po7][po8] - qcef
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = (qubits+1)*i + l1
                    po2 = (qubits+1)*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef


# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range((qubits+1)*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",(qubits+1)*Dimension,"','q",(qubits+1)*Dimension,"'):",format(QM[(qubits+1)*Dimension-1][(qubits+1)*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range((qubits+1)*Dimension-1):
    for j in range(i+1,(qubits+1)*Dimension):
        qdrt1 = i + 1
        qdrt2 = j + 1
        if i == (qubits+1)*Dimension-2 and j == (qubits+1)*Dimension-1:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
        else:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[-1.4000e+01  8.7200e+02  1.7440e+03 -3.4880e+03 -2.4800e+02 -4.9600e+02
  -9.9200e+02  1.9840e+03  3.8000e+01  7.6000e+01  1.5200e+02 -3.0400e+02
  -1.6000e+02 -3.2000e+02 -6.4000e+02  1.2800e+03]
 [ 0.0000e+00  4.0800e+02  3.4880e+03 -6.9760e+03 -4.9600e+02 -9.9200e+02
  -1.9840e+03  3.9680e+03  7.6000e+01  1.5200e+02  3.0400e+02 -6.0800e+02
  -3.2000e+02 -6.4000e+02 -1.2800e+03  2.5600e+03]
 [ 0.0000e+00  0.0000e+00  2.5600e+03 -1.3952e+04 -9.9200e+02 -1.9840e+03
  -3.9680e+03  7.9360e+03  1.5200e+02  3.0400e+02  6.0800e+02 -1.2160e+03
  -6.4000e+02 -1.2800e+03 -2.5600e+03  5.1200e+03]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.5808e+04  1.9840e+03  3.9680e+03
   7.9360e+03 -1.5872e+04 -3.0400e+02 -6.0800e+02 -1.2160e+03  2.4320e+03
   1.2800e+03  2.5600e+03  5.1200e+03 -1.0240e+04]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -3.4800e+02  8.1600e+02
   1.6320e+03 -3.2640e+03  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  -4.8000e+01 -9.6000e+01 -1.9200e+02  3.8

In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-14.0, ('q2','q2'):408.0, ('q3','q3'):2560.0, ('q4','q4'):15808.0, ('q5','q5'):-348.0, ('q6','q6'):-288.0, ('q7','q7'):1056.0, ('q8','q8'):17472.0, ('q9','q9'):1112.0, ('q10','q10'):2612.0, ('q11','q11'):6776.0, ('q12','q12'):5072.0, ('q13','q13'):1693.0, ('q14','q14'):3856.0, ('q15','q15'):9592.0, ('q16','q16'):3376.0}

quadratic = {('q1','q2'):872.0, ('q1','q3'):1744.0, ('q1','q4'):-3488.0, ('q1','q5'):-248.0, ('q1','q6'):-496.0, ('q1','q7'):-992.0, ('q1','q8'):1984.0, ('q1','q9'):38.0, ('q1','q10'):76.0, ('q1','q11'):152.0, ('q1','q12'):-304.0, ('q1','q13'):-160.0, ('q1','q14'):-320.0, ('q1','q15'):-640.0, ('q1','q16'):1280.0, ('q2','q3'):3488.0, ('q2','q4'):-6976.0, ('q2','q5'):-496.0, ('q2','q6'):-992.0, ('q2','q7'):-1984.0, ('q2','q8'):3968.0, ('q2','q9'):76.0, ('q2','q10'):152.0, ('q2','q11'):304.0, ('q2','q12'):-608.0, ('q2','q13'):-320.0, ('q2','q14'):-640.0, ('q2','q15'):-1280.0, ('q2','q16'):2560.0, ('q3','q4'):-13952.0, ('q3','q5'):-992.0, ('q3','q6'):-1984.0, ('q3','q7'):-3968.0, ('q3','q8'):7936.0, ('q3','q9'):152.0, ('q3','q10'):304.0, ('q3','q11'):608.0, ('q3','q12'):-1216.0, ('q3','q13'):-640.0, ('q3','q14'):-1280.0, ('q3','q15'):-2560.0, ('q3','q16'):5120.0, ('q4','q5'):1984.0, ('q4','q6'):3968.0, ('q4','q7'):7936.0, ('q4','q8'):-15872.0, ('q4','q9'):-304.0, ('q4','q10'):-608.0, ('q4','q11'):-1216.0, ('q4','q12'):2432.0, ('q4','q13'):1280.0, ('q4','q14'):2560.0, ('q4','q15'):5120.0, ('q4','q16'):-10240.0, ('q5','q6'):816.0, ('q5','q7'):1632.0, ('q5','q8'):-3264.0, ('q5','q9'):0.0, ('q5','q10'):0.0, ('q5','q11'):0.0, ('q5','q12'):0.0, ('q5','q13'):-48.0, ('q5','q14'):-96.0, ('q5','q15'):-192.0, ('q5','q16'):384.0, ('q6','q7'):3264.0, ('q6','q8'):-6528.0, ('q6','q9'):0.0, ('q6','q10'):0.0, ('q6','q11'):0.0, ('q6','q12'):0.0, ('q6','q13'):-96.0, ('q6','q14'):-192.0, ('q6','q15'):-384.0, ('q6','q16'):768.0, ('q7','q8'):-13056.0, ('q7','q9'):0.0, ('q7','q10'):0.0, ('q7','q11'):0.0, ('q7','q12'):0.0, ('q7','q13'):-192.0, ('q7','q14'):-384.0, ('q7','q15'):-768.0, ('q7','q16'):1536.0, ('q8','q9'):0.0, ('q8','q10'):0.0, ('q8','q11'):0.0, ('q8','q12'):0.0, ('q8','q13'):384.0, ('q8','q14'):768.0, ('q8','q15'):1536.0, ('q8','q16'):-3072.0, ('q9','q10'):776.0, ('q9','q11'):1552.0, ('q9','q12'):-3104.0, ('q9','q13'):306.0, ('q9','q14'):612.0, ('q9','q15'):1224.0, ('q9','q16'):-2448.0, ('q10','q11'):3104.0, ('q10','q12'):-6208.0, ('q10','q13'):612.0, ('q10','q14'):1224.0, ('q10','q15'):2448.0, ('q10','q16'):-4896.0, ('q11','q12'):-12416.0, ('q11','q13'):1224.0, ('q11','q14'):2448.0, ('q11','q15'):4896.0, ('q11','q16'):-9792.0, ('q12','q13'):-2448.0, ('q12','q14'):-4896.0, ('q12','q15'):-9792.0, ('q12','q16'):19584.0, ('q13','q14'):940.0, ('q13','q15'):1880.0, ('q13','q16'):-3760.0, ('q14','q15'):3760.0, ('q14','q16'):-7520.0, ('q15','q16'):-15040.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q2 q3 q4 q5 q6 q7 q8 q9  energy num_oc. ...
4    1   1   1   1   0   1   1   1  0  0  0  0  1  0  0  1 -2412.0       1 ...
5    0   1   1   1   0   1   1   1  0  0  0  1  0  0  0  1 -2340.0       1 ...
6    0   1   0   0   1   1   0   1  1  1  1  0  0  0  0  0 -2279.0       1 ...
8    0   0   0   0   1   0   1   1  0  0  0  1  0  0  0  1 -2269.0       1 ...
9    0   1   0   0   1   1   0   1  1  1  1  1  1  1  1  0 -2259.0       6 ...
11   0   0   0   0   0   0   1   1  0  0  0  1  0  0  0  0 -2228.0       1 ...
12   0   0   0   0   0   0   1   1  0  0  0  0  0  0  0  1 -2184.0       1 ...
13   0   0   0   0   1   1   0   1  1  1  1  1  1  1  1  1 -2153.0       5 ...
14   0   1   0   0   0   1   0   1  1  1  1  1  1  1  1  1 -2136.0       2 ...
15   0   0   1   1   0   0   0   0  1  0  0  1  1  0  0  1 -2136.0       3 ...
16   0   1   1   1   0   1   1   1  1  0  0  1  1  0  0  1 -2136.0       1 ...
17   0   0   0   0   0   0   1   1  1  1  1  1  1  1